In [ ]:
# import python's module
import numpy as np
import pandas as pd
import json

In [ ]:
# load data
log_df = pd.read_csv("operation-log.csv").replace(np.nan, None)
log_df

In [ ]:
operators = []
for person in log_df['operator'].unique():
    operators.append({
        "firstname": person,
        "lastname": "",
        "phone": ""
    })
operator_df = pd.DataFrame(operators)

operators = []
for index, row in operator_df.iterrows():
    operators.append({
        "model": "acrelog.operator", 
        "pk": index+1,
        "fields": {
            "firstname": row['firstname']
        }
    })
operators

In [ ]:
with open('operator-data.json', 'w') as fp:
    json.dump(operators, fp=fp, indent=4)

In [ ]:
locations = []
for field in log_df['location'].unique():
    locations.append({
        "name": field,
        "zipcode": "",
    })
location_df = pd.DataFrame(locations)

locations = []
for index, row in location_df.iterrows():
    locations.append({
        "model": "acrelog.field", 
        "pk": index+1,
        "fields": {
            "name": row['name']
        }
    })
locations

In [ ]:
with open('location-data.json', 'w') as fp:
    json.dump(locations, fp=fp, indent=4)

In [ ]:
machines = []
for machine in log_df['Power Unit'].unique():
    if machine in [None, 'Not applicable']: 
        continue
    machines.append({
        "name": machine
    })
machine_df = pd.DataFrame(machines)

machines = []
for index, row in machine_df.iterrows():
    machines.append({
        "model": "acrelog.machine", 
        "pk": index+1,
        "fields": {
            "name": row['name']
        }
    })
machines

In [ ]:
with open('machine-data.json', 'w') as fp:
    json.dump(machines, fp=fp, indent=4)

In [ ]:
seeds = []
for seed in log_df['Seeds planted'].unique():
    if seed in [None]: 
        continue
    plant, genotype = seed.split(' - ')
    seeds.append({
        "plant": plant,
        "genotype": genotype,
        "name": seed
    })
seed_df = pd.DataFrame(seeds)

seeds = []
for index, row in seed_df.iterrows():
    seeds.append({
        "model": "acrelog.seed", 
        "pk": index+1,
        "fields": {
            "plant": row['plant'],
            "genotype": row['genotype']
        }
    })
seeds

In [ ]:
with open('seed-data.json', 'w') as fp:
    json.dump(seeds, fp=fp, indent=4)

In [ ]:
sub_log_df = log_df.copy()
sub_log_df

In [ ]:
sub_log_df['Fertilizers applied'] = log_df['Fertilizers applied'].apply(lambda x: ", ".join(sorted(x.split(","))) if x else None)
sub_log_df

In [ ]:
OPERATION_TYPES = [
    ("tillage", "Tillage"),
    ("plant", "Plant"),
    ("spray", "Spread/Spray"),
    ("soil", "Soil Sampled"),
    ("harvest", "Harvest"),
    ("other", "Other"),
]

operation_type = {}
for x in OPERATION_TYPES:
    operation_type[x[1]] = x[0]
operation_type

In [ ]:
sub_log_df['operator'] = log_df['operator'].map(dict((v, k+1) for k, v in operator_df.to_dict()['firstname'].items()))
sub_log_df

In [ ]:
sub_log_df['location'] = log_df['location'].map(dict((v, k+1) for k, v in location_df.to_dict()['name'].items()))
sub_log_df

In [ ]:
sub_log_df['Power Unit'] = log_df['Power Unit'].map(dict((v, k+1) for k, v in machine_df.to_dict()['name'].items()))
sub_log_df['Power Unit'] = sub_log_df['Power Unit'].replace(np.nan, None)
sub_log_df

In [ ]:
sub_log_df['Seeds planted'] = log_df['Seeds planted'].map(dict((v, k+1) for k, v in seed_df.to_dict()['name'].items()))
sub_log_df['Seeds planted'] = sub_log_df['Seeds planted'].replace(np.nan, None)
sub_log_df

In [ ]:
sub_log_df['operation'] = log_df['operation'].map(operation_type)
sub_log_df

In [ ]:
sub_log_df[~sub_log_df['Seeds planted'].isna()]

In [ ]:
sub_log_df['date'] = pd.to_datetime(sub_log_df['date']).dt.strftime('%Y-%m-%d')

In [ ]:
logs = []
for index, row in sub_log_df.iterrows():
    logs.append({
        "model": "acrelog.operation", 
        "pk": index+1,
        "fields": {
            "date": row['date'],
            "location": row['location'],
            "operator": row['operator'],
            "operation_type": row['operation'],
            "note": row['note'],
            "tool": row['Power Unit'],
            "seed": row['Seeds planted'],
            "seed_rate": row['Seeding Rate (seeds/ac)'],
            "fertillizer": row['Fertilizers applied'],
            "fertiliizer_rate": row['Fertilizer Rate (lb/ac)']
        }
    })
logs

In [ ]:
with open('log-data.json', 'w') as fp:
    json.dump(logs, fp=fp, indent=4)